In [24]:
# Import libraries
import pandas as pd
from IPython.display import display
%matplotlib inline

In [25]:
# Read student data
loan_data = pd.read_csv("loan_data.csv")
print ("Loan data read successfully!")

Loan data read successfully!


In [26]:
display(loan_data.head())
print(loan_data.dtypes)

,funded_amnt,term,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_util,collections_12_mths_ex_med,mths_since_last_major_derog
0,5000.0,0,162.87,10,RENT,24000.0,Verified,0,credit_card,27.65,0.0,1.0,180.0,180.0,3.0,0.0,83.7,0.0,180.0
1,2500.0,1,59.83,0,RENT,30000.0,Source Verified,1,car,1.00,0.0,5.0,180.0,180.0,3.0,0.0,9.4,0.0,180.0
2,2400.0,0,84.33,10,RENT,12252.0,Not Verified,0,small_business,8.72,0.0,2.0,180.0,180.0,2.0,0.0,98.5,0.0,180.0
3,10000.0,0,339.31,10,RENT,49200.0,Source Verified,0,other,20.00,0.0,1.0,35.0,180.0,10.0,0.0,21.0,0.0,180.0
4,5000.0,0,156.46,3,RENT,36000.0,Source Verified,0,wedding,11.20,0.0,3.0,180.0,180.0,9.0,0.0,28.3,0.0,180.0


funded_amnt                    float64
term                             int64
installment                    float64
emp_length                       int64
home_ownership                  object
annual_inc                     float64
verification_status             object
loan_status                      int64
purpose                         object
dti                            float64
delinq_2yrs                    float64
inq_last_6mths                 float64
mths_since_last_delinq         float64
mths_since_last_record         float64
open_acc                       float64
pub_rec                        float64
revol_util                     float64
collections_12_mths_ex_med     float64
mths_since_last_major_derog    float64
dtype: object


In [27]:
# TODO: Calculate number of records
n_loans = float(len(loan_data))

# TODO: Calculate number of features
n_features = float(len(loan_data.columns) - 1)

# TODO: Calculate paid in full loans
n_pif = float(sum(1 for item in loan_data['loan_status'] if item==(0)))

# TODO: Calculate charged off loans
n_chargeoff = float(sum(1 for item in loan_data['loan_status'] if item==(1)))

# TODO: Calculate successful loan rate
success_rate = n_pif / n_loans * 100

# Print the results
print("Total number of loans: {}".format(n_loans))
print("Number of features: {}".format(n_features))
print("Number of loans paid in full: {}".format(n_pif))
print("Number of charge-offs: {}".format(n_chargeoff))
print("Success rate of loans: {:.2f}%".format(success_rate))

Total number of loans: 252535.0
Number of features: 18.0
Number of loans paid in full: 207374.0
Number of charge-offs: 45161.0
Success rate of loans: 82.12%


In [28]:
# Store the 'loan_status' feature in a new variable and remove it from the dataset
outcomes = loan_data['loan_status']
data = loan_data.drop(['loan_status'], axis = 1)

# Show the new dataset with 'loan_status' removed
display(data.head())
display(outcomes.head())

,funded_amnt,term,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_util,collections_12_mths_ex_med,mths_since_last_major_derog
0,5000.0,0,162.87,10,RENT,24000.0,Verified,credit_card,27.65,0.0,1.0,180.0,180.0,3.0,0.0,83.7,0.0,180.0
1,2500.0,1,59.83,0,RENT,30000.0,Source Verified,car,1.00,0.0,5.0,180.0,180.0,3.0,0.0,9.4,0.0,180.0
2,2400.0,0,84.33,10,RENT,12252.0,Not Verified,small_business,8.72,0.0,2.0,180.0,180.0,2.0,0.0,98.5,0.0,180.0
3,10000.0,0,339.31,10,RENT,49200.0,Source Verified,other,20.00,0.0,1.0,35.0,180.0,10.0,0.0,21.0,0.0,180.0
4,5000.0,0,156.46,3,RENT,36000.0,Source Verified,wedding,11.20,0.0,3.0,180.0,180.0,9.0,0.0,28.3,0.0,180.0


0    0
1    1
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [29]:
display(data.describe())

,funded_amnt,term,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_util,collections_12_mths_ex_med,mths_since_last_major_derog
count,252535.000000,252535.000000,252535.000000,252535.000000,2.525350e+05,252535.000000,252535.000000,252535.000000,252535.000000,252535.000000,252535.000000,252535.000000,252535.000000,252535.000000,252535.000000
mean,13527.641416,0.222809,418.249895,5.563973,7.253306e+04,16.549335,0.249811,0.852666,115.824844,166.991130,10.929966,0.143442,54.314660,0.006736,154.312119
std,8108.755794,0.416132,244.902930,3.788244,5.879248e+04,7.786736,0.740532,1.066970,73.384033,35.981015,4.812420,0.432747,24.730282,0.089072,54.194762
min,500.000000,0.000000,15.690000,-1.000000,3.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,7200.000000,0.000000,239.560000,2.000000,4.500000e+04,10.760000,0.000000,0.000000,37.000000,180.000000,7.000000,0.000000,36.300000,0.000000,180.000000
50%,12000.000000,0.000000,365.230000,6.000000,6.200000e+04,16.200000,0.000000,1.000000,180.000000,180.000000,10.000000,0.000000,55.800000,0.000000,180.000000
75%,18125.000000,0.000000,547.550000,10.000000,8.700000e+04,22.000000,0.000000,1.000000,180.000000,180.000000,14.000000,0.000000,73.900000,0.000000,180.000000
max,35000.000000,1.000000,1424.570000,10.000000,8.706582e+06,57.140000,29.000000,8.000000,180.000000,180.000000,30.000000,7.000000,150.000000,6.000000,180.000000


In [30]:
data = pd.get_dummies(data, columns=['home_ownership', 'purpose', 'verification_status'])

In [31]:
display(data.head())

,funded_amnt,term,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,...,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,5000.0,0,162.87,10,24000.0,27.65,0.0,1.0,180.0,180.0,...,0,0,0,0,0,0,0,0,0,1
1,2500.0,1,59.83,0,30000.0,1.00,0.0,5.0,180.0,180.0,...,0,0,0,0,0,0,0,0,1,0
2,2400.0,0,84.33,10,12252.0,8.72,0.0,2.0,180.0,180.0,...,0,0,0,0,1,0,0,1,0,0
3,10000.0,0,339.31,10,49200.0,20.00,0.0,1.0,35.0,180.0,...,0,0,1,0,0,0,0,0,1,0
4,5000.0,0,156.46,3,36000.0,11.20,0.0,3.0,180.0,180.0,...,0,0,0,0,0,0,1,0,1,0


In [32]:
print(data.dtypes)

funded_amnt                            float64
term                                     int64
installment                            float64
emp_length                               int64
annual_inc                             float64
dti                                    float64
delinq_2yrs                            float64
inq_last_6mths                         float64
mths_since_last_delinq                 float64
mths_since_last_record                 float64
open_acc                               float64
pub_rec                                float64
revol_util                             float64
collections_12_mths_ex_med             float64
mths_since_last_major_derog            float64
home_ownership_MORTGAGE                  uint8
home_ownership_OWN                       uint8
home_ownership_RENT                      uint8
purpose_car                              uint8
purpose_credit_card                      uint8
purpose_debt_consolidation               uint8
purpose_educa